In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from model import model
from DataExtraction import utils
import os

In [ ]:
# Load, from each model, the results, the model configuration and the evaluation metrics
os.chdir(utils.get_root_dir() + '/model/final_models')
n_models = len([name for name in os.listdir('.') if 'model_' in name])
training_stats, testing_stats, model_results, model_info = {}, {}, {}, {}
for i in range(n_models):
    print(f'Loading model {i}')
    m = model.load_model(f'model_{i}')
    model_info[m.model_name] = m.model_config.variables
    training_stats[m.model_name] = pd.read_csv(m.model_config.model_src + '/train/training_stats.csv')
    testing_stats[m.model_name] = pd.read_csv(m.model_config.model_src + '/test/losses.csv')
    model_results[m.model_name] = np.load(m.model_config.model_src + '/test/y_pred.npy'), np.load(m.model_config.model_src + '/test/y_test.npy')
    del m